In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [6]:
targets = train.target.value_counts(sort=True)
targets

target
-0.159740      3477
-0.759959      3471
-0.050068      3470
-0.079870      3467
 1.109838      3459
               ... 
 62.299965        1
 59.139730        1
-70.880060        1
-108.349920       1
-72.960260        1
Name: count, Length: 15934, dtype: int64

In [7]:
import importlib

In [8]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()
model = torch_classes.GRUNet(12,hidden_size)
X = trading_data.packed_x[0]
Y = trading_data.packed_y[0]

 10%|▉         | 19/200 [00:09<01:32,  1.95it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:48<00:43,  2.30it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:05<01:02,  1.10it/s]

...

 79%|███████▉  | 158/200 [01:23<00:25,  1.63it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [02:36<00:00,  1.28it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 7988.99it/s]
c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
trading_df = trading_data

In [10]:
model = torch_classes.GRUNet(12,hidden_size).to('cuda:0')

In [11]:
stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[0]]
stock_data = trading_df.train_batches
X = trading_data.packed_x[0]
Y = trading_data.packed_y[0].data

hidden_in = torch.stack([x.hidden for x in stocks])

output,hidden = model(X)
hidden = hidden.transpose(0,1)
output  = torch.flatten(output)

[setattr(obj, 'hidden', val) for obj, val in zip(stocks,hidden)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[0]]
len(stocks)

191

In [13]:
torch.zeros(31,1).shape

torch.Size([31, 1])

In [14]:
hidden.shape

torch.Size([191, 1, 64])

In [15]:
trading_df.stocksDict[0].hidden.shape

torch.Size([1, 64])

In [16]:
trading_df.reset_hidden()

In [17]:
trading_df.stocksDict[0].hidden.shape

torch.Size([1, 32])

In [18]:
def train_model(trading_df:torch_classes.TradingData, model:torch_classes.GRUNet, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
            wandb.define_metric("val_epoch_loss_l1", summary="min")
            wandb.define_metric("epoch_l1_loss", summary="min")
            example_ct = 0
            epochs = 10000
            num_batches = len(trading_df.train_batches)-2
            criterion = nn.SmoothL1Loss()
            reg_L1 = nn.L1Loss()
            model = model.to('cuda:0')
            optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
            trading_df.reset_hidden(config['hidden_size'])
            for epoch in trange(epochs):
                model.train()
                loss_list = []
                
                for i in range(0,384):
                    # print(i)

                    stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[i]] #Stocks for the Day
                    stock_data = trading_df.train_batches

                    example_ct+=1

                    X = trading_data.packed_x[i]
                    Y = trading_data.packed_y[i].data

                    hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

                    output,hidden = model(X,hidden_in)
                    hidden = hidden.transpose(0,1)
                    output  = torch.flatten(output)

                    [setattr(obj, 'hidden', val.detach()) for obj, val in zip(stocks,hidden)]

                    if output.shape!=Y.shape:
                         print(i,output.shape,Y.shape,)

                    loss = criterion(output,Y)
                    # print(loss)

                    loss.backward()
                    loss_list.append((i,loss))
                    L1_loss = reg_L1(output,Y)
                    if loss.isnan():
                         continue
                    # print(loss)
                    optimizer.step()
                    if i == 0:
                        epoch_loss = loss
                        epoch_reg_l1 = L1_loss
                        # print(epoch_loss)
                    else:
                        if loss.isnan():
                            pass
                        epoch_loss = loss+epoch_loss
                        epoch_reg_l1 = L1_loss+epoch_reg_l1

                    wandb.log({"loss_1": torch.mean(loss).item()})
                    trading_df.detach_hidden()
                wandb.log({"epoch_loss": epoch_loss/384, "epoch_l1_loss": epoch_reg_l1/384, 'epoch':epoch})
                validate_model(trading_df,model,criterion,epoch)
                if epoch%10==0:
                    
                    trading_df.create_hidden_states_dict_v2()
                    model_saver(model,optimizer,epoch,0,0,trading_df.train_hidden_dict)
                    
                trading_df.reset_hidden(hidden_size=config['hidden_size'])
                # print(epoch_loss)
                # print(loss_list)
              

@torch.no_grad()          
def validate_model(trading_df:torch_classes.TradingData,model:torch_classes.GRUNet,criterion,epoch):
    model.eval()
    val_batches = trading_df.packed_val_x
    len_val = len(val_batches)
    loss_list = []

    # print(len_val)

    for i in range(0,len_val-1):
        # print(i)
        stocks = [trading_df.stocksDict[x] for x in trading_df.val_stock_batches[i]] 
         
        X = trading_df.packed_val_x[i]
        Y = trading_df.packed_val_y[i].data

        hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

        output,hidden = model(X,hidden_in)
        hidden = hidden.transpose(0,1)
        output  = torch.flatten(output)

        # [setattr(obj, 'hidden', val) for obj, val in zip(stocks,hidden)]

        loss = criterion(output,Y)

        loss_list.append((i,loss))

        # if loss.isnan():
        #     continue

        # print(f"{loss=}")
        # print(f"{i=}")

        if i == 0:
            epoch_loss = loss
            # print(epoch_loss)
        else:
            # print(loss)
            if loss.isnan():
                continue
            epoch_loss = loss+epoch_loss
    trading_df.reset_hidden(64)
    wandb.log({"val_epoch_loss": epoch_loss/len_val, 'epoch':epoch})
    





In [19]:
trading_df.detach_hidden()
trading_df.reset_hidden(hidden_size=hidden_size)

In [20]:
config_static = {'learning_rate':0.00002, 'hidden_size':64}

In [21]:
print('done')

done


In [22]:
len_data = len(trading_data.daysDict)
len_validation = int(len(trading_data.daysDict)*0.2)
len_train = len_data-len_validation
val_range = range(len_train+1,len_data)

In [23]:
print(len_data,len_validation,len_train,val_range)

481 96 385 range(386, 481)


In [24]:
val_batches = trading_df.packed_val_x
len_val = len(val_batches)
loss_list = []

print(len_val)

95


In [25]:
CUDA_LAUNCH_BLOCKING=1
train_model(trading_data,model,config=config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


  0%|          | 0/10000 [00:00<?, ?it/s]

created path
